In [ ]:


import streamlit as st
import requests
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

In [ ]:
load_dotenv()

AVAL_API_KEY = os.getenv("AVAL_API_KEY")
WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")

In [ ]:


client = OpenAI(
    api_key=AVAL_API_KEY,
    base_url="https://api.avalai.ir/v1"
)

In [ ]:


def get_weather(city):
    if not city:
        return "نامشخص"

    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": WEATHER_API_KEY,
        "units": "metric",
        "lang": "fa"
    }

    r = requests.get(url, params=params)
    data = r.json()

    if r.status_code != 200:
        return "اطلاعات آب‌وهوا در دسترس نیست"

    return f"{data['weather'][0]['description']}، دما {data['main']['temp']} درجه"

In [ ]:


def travel_agent(prompt):
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "تو یک دستیار برنامه‌ریز سفر هستی."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [ ]:


st.set_page_config(page_title="دستیار سفر", layout="centered")
st.title("✈️ دستیار هوشمند برنامه‌ریزی سفر")

origin = st.text_input("مبدا سفر")
destination = st.text_input("مقصد سفر")

budget = st.number_input("بودجه (تومان)", min_value=0)
days = st.number_input("تعداد روز سفر", min_value=1)
travel_type = st.selectbox("نوع سفر", ["هواپیما", "قطار", "زمینی"])

In [ ]:


if st.button("ساخت برنامه سفر"):
    weather = get_weather(destination)

    prompt = f"""
    مبدا: {origin}
    مقصد: {destination}
    بودجه: {budget} تومان
    مدت: {days} روز
    نوع سفر: {travel_type}
    وضعیت آب‌وهوا: {weather}

    یک برنامه سفر پیشنهادی با هزینه تقریبی ارائه بده.
    """

    result = travel_agent(prompt)
    st.text_area("برنامه پیشنهادی سفر", result, height=300)

In [ ]:
streamlit run final_project/app.py

#you should run this code in app.py